## Practical uses of fixed-point combinators

Presentation to the San Diego Python User Group on 2021-01-28

This presentation is available at https://github.com/ecolban/SDPUG/fixed_point_combinators

## Problem statement

Assume I want to log calls to a function. In the log, I want the time of the call, the name of the function called and the values of the arguments passed in the call. For this purpose, I can define a higher order function that I apply to the function called.

In [ ]:
from functools import wraps
from datetime import datetime

def log(f):
    @wraps(f)
    def h(*args, **kwargs):
        all_args = [repr(a) for a in args]
        all_args += [f"{k}={repr(v)}" for k, v in kwargs.items()]
        print(f"{datetime.utcnow()}: {f.__name__}({', '.join(all_args)})")
        return f(*args, **kwargs)

    return h


To illustrate this log function, I can apply it to a call to `re.match`:

In [ ]:
import re

pattern = r"\d{4}-\d{2}-\d{2}"
date = "2021-01-13T18:36:21"
re.match(pattern, date)

In [ ]:
log(re.match)(pattern, date)

Let's define a Fibonacci function and apply `log` to that function:

In [ ]:
def fib0(n):
    return 1 if n < 2 else fib0(n - 1) + fib0(n - 2)

In [ ]:
log(fib0)(5)

Notice that the recursive calls to fib are not logged. If I want to log _every_ call to `fib`, I can use the `log` function as a decorator:

In [ ]:
@log
def fib0(n):
    return 1 if n < 2 else fib0(n - 1) + fib0(n - 2)

In [ ]:
fib0(5)

But it may be impractical to log _every_ call. If I were to log a call to `fib(20)`, I would get pages and pages of logs. What I want is:
+ to selectively log certain calls of `fib`, but when I do so, I want the recursive calls to be logged too.
+ a general solution that can be applied to other recursive functions

## Another way of writing recursive functions

### Example 1: Fibonacci

Let's redefine a Fibonacci function while trying to avoid recursion:

In [ ]:
def fib_inner(n):
    return 1 if n < 2 else f(n - 1) + f(n - 2)

Notice that I have replaced the recursive calls to `fib_inner` with calls to a free variable `f`. To bind this variable, I place the definition of `fib_inner` inside a the definition of a function `fib_outer`, which takes `f` as argument, and hence, binds `f`:

In [ ]:
def fib_outer(f):
    def fib_inner(n):
        return 1 if n < 2 else f(n - 1) + f(n - 2)
    
    return fib_inner

Now, all I need to do is to call `fib_outer` with the "correct" value of `f` so that `fib_outer` returns the Fibonacci function. But, which value of `f` is that?

Calling `fib_outer` with `fib_inner` as the value of `f` would make `fib_inner` a recursive implementation of the Fibonacci function. But `fib_inner` is local to `fib_outer` and not accessible outside `fib_outer`. 

But `fib_inner` is returned by `fib_outer`, so the "correct" function to pass to `fib_outer` is the function that `fib_outer` would return if `fib_outer` were called with that function as argument! 🤔

If we call the "correct" function `fib`, we have:

In [ ]:
fib = fib_outer(fib)

Let's re-write the right-hand as a lambda expression.

In [ ]:
fib = lambda n: fib_outer(fib)(n)

In [ ]:
[fib(n) for n  in range(11)]

Magic!!

What's happened is that `fib = lambda n: fib_outer(fib)(n)` is not a regular assignment, but a recursive function definition. We can re-write this as a `def`:

In [ ]:
def fib(n):
    return fib_outer(fib)(n)

Whenever we have a function $F$ and a value $x$ such that $F(x) = x$, then $x$ is said to be a _fixed-point_ of $F$. Notice that `fib` is a fixed-point of `fib_outer`.

In [ ]:
all(fib(n) == fib_outer(fib)(n) for n in range(20))

### Example 2: Factorial

Let's repeat this exercise with another function: _factorial_

First, define `fact_outer` and `fact_inner`:

In [ ]:
def fact_outer(f):
    def fact_inner(n):
        return 1 if n == 0 else n * f(n - 1)

    return fact_inner

Then define `fact` as the fixed-point of `fact_outer`:

In [ ]:
def fact(n):
    return fact_outer(fact)(n)

In [ ]:
[fact(n) for n in range(11)]

## Fixed-point combinators

Notice the similarity of definitions of `fib` and `fact`. Besides naming (where one says `fib` the other says `fact`; where one says `fib_outer` the other says `fact_outer`), they are identical. We can extract a function that returns both:

In [ ]:
def fixed_point(outer):
    def fp(*args, **kwargs):
        return outer(fp)(*args, **kwargs)
    return fp

(If you wonder what the _y-combinator_ is, it's just the fixed-point combinator written in lambda calculus.)

With this definition at hand, we could redefine `fib` and `fact` as:

In [ ]:
fib = fixed_point(fib_outer)
fact = fixed_point(fact_outer)

In [ ]:
[fib(n) for n in range(11)]

In [ ]:
[fact(n) for n in range(11)]

### Fixed-point combinator as a decorator

In [ ]:
def fixed_point(outer):
    @wraps(outer)
    def fp(*args, **kwargs):
        return outer(fp)(*args, **kwargs)
    return fp

In [ ]:
@fixed_point
def fib(f):
    def fib_inner(n):
        return 1 if n < 2 else f(n - 1) + f(n - 2)
    
    return fib_inner

In [ ]:
[fib(n) for n in range(11)]

In [ ]:
fib.__name__

## A multitude of fixed-points

A function like `fib_outer` may have several fixed-points, which, although functionally the same, can vary in side-effects and efficiency. They can be obtained by applying different fixed-point combinators. Here are some examples.

### Example 1: A fixed-point that logs

In [ ]:
def logging_fixed_point(outer):
    @log
    @wraps(outer)
    def fp(*args, **kwargs):
        return outer(fp)(*args, **kwargs)
    return fp

In [ ]:
logging_fixed_point(fib_outer)(5)

### Example 2: A fixed-point that memoizes

In [ ]:
from functools import lru_cache

def memoizing_fixed_point(outer):
    @lru_cache
    def fp(*args, **kwargs):
        return outer(fp)(*args, **kwargs)
    return fp

In [ ]:
fixed_point(fib_outer)(34)

In [ ]:
memoizing_fixed_point(fib_outer)(100)

### Example 3: A fixed-point that logs and memoizes

In [ ]:
def logging_memoizing_fixed_point(outer):

    @log
    @lru_cache
    @wraps(outer)
    def fp(*args, **kwargs):
        return outer(fp)(*args, **kwargs)
    return fp

In [ ]:
logging_memoizing_fixed_point(fib_outer)(10)

## Conclusion

When you define a recursive function by using a fixed-point combinator on an "outer" function as illustrated in this presentation, you can vary the fixed-point combinator for selected calls. A fixed-point combinator can be thought of as a kind of "dynamic decorator".